In [92]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

In [93]:
df = pd.read_csv('merged_nifty50_data.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values('Date', inplace=True)

In [94]:
selected_stocks = ['TCS', 'INFY', 'HCLTECH', 'WIPRO', 'TECHM', 'LT', 
                   'BHARTIARTL', 'ICICIBANK', 'HDFCBANK', 'AXISBANK', 
                   'KOTAKBANK', 'SBIN', 'INDUSINDBK', 'YESBANK', 'TATAMOTORS']


In [95]:
df = df[df['Symbol'].isin(selected_stocks)]

In [96]:
model = load_model('model.keras')

In [97]:
app = dash.Dash(__name__)

In [98]:
app.layout = html.Div([
    html.H1('Stock Price Prediction'),
    dcc.Dropdown(
        id='stock-dropdown',
        options=[{'label': stock, 'value': stock} for stock in selected_stocks],
        value='TCS'
    ),
    dcc.Graph(id='actual-graph'),
    dcc.Graph(id='predicted-graph')
])

In [99]:
@app.callback(
    Output('actual-graph', 'figure'),
    [Input('stock-dropdown', 'value')]
)
def update_actual_graph(selected_stock):
    stock_data = df.loc[df['Symbol'] == selected_stock].copy()
    fig = go.Figure(data=[go.Scatter(x=stock_data['Date'], y=stock_data['Close'], name='Actual', line=dict(color='blue'))])
    return fig

@app.callback(
    Output('predicted-graph', 'figure'),
    [Input('stock-dropdown', 'value')]
)
def update_predicted_graph(selected_stock):
    stock_data = df.loc[df['Symbol'] == selected_stock].copy()
    stock_data.loc[:, 'MA_10'] = stock_data['Close'].rolling(window=10).mean()
    stock_data.loc[:, 'MA_50'] = stock_data['Close'].rolling(window=50).mean()
    stock_data.loc[:, 'EMA_10'] = stock_data['Close'].ewm(span=10, adjust=False).mean()
    delta = stock_data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    stock_data.loc[:, 'RSI'] = 100 - (100 / (1 + rs))
    stock_data.dropna(inplace=True)  # Ensure no NaN values are present
    features = ['Close', 'MA_10', 'MA_50', 'EMA_10', 'RSI']
    data = stock_data[features].values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    seq_length = 60
    x, _ = create_sequences(scaled_data, seq_length)
    x = np.reshape(x, (x.shape[0], x.shape[1], 5))  # Ensure the shape matches the training shape
    predictions = model.predict(x)
    predictions = scaler.inverse_transform(np.concatenate((predictions, np.zeros((predictions.shape[0], 4))), axis=1))[:, 0]  # Inverse transform only the 'Close' predictions
    fig = go.Figure(data=[
        go.Scatter(x=stock_data['Date'][-len(predictions):], y=predictions, name='Predicted', line=dict(color='red')),
        go.Scatter(x=stock_data['Date'], y=stock_data['Close'], name='Actual', line=dict(color='blue'))
    ])
    return fig

def create_sequences(data, seq_length):
    x, y = [], []
    for i in range(len(data) - seq_length):
        x.append(data[i:i + seq_length])
        y.append(data[i + seq_length, 0])  # only take the Close value
    return np.array(x), np.array(y)

In [100]:
if __name__ == '__main__':
    app.run_server()

256/256 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step 
298/298 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step
